In [1]:
from david_cnn import *
os.environ['XDG_CACHE_HOME']='/tmp/xdg-cache'


In [2]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.xavier_normal_(m.weight.data)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        #nn.init.xavier_normal_(m.weight.data)
        nn.init.constant_(m.bias.data, 0)

In [3]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

net=Nnet().to(computing_device)
net.apply(weights_init)

# Print the model
print(net)

#loss criteria are defined in the torch.nn package
criterion = nn.CrossEntropyLoss()


#Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(net.parameters(),lr = 0.001)

CUDA is supported
Nnet(
  (main): Sequential(
    (0): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU(inplace=True)
    (2): Conv2d(18, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(30, 35, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (6): Conv2d(35, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(40, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): Conv2d(45, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (12): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [5]:
transform = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor()])
dataset = loader('train.csv','/datasets/cs154-fa19-public/',transform=transform)
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:

# Addressing data label imbalance issue through resampling 
dataframe = dataset.frame
X = dataset.frame['path'] # Path names (X)
y = dataset.frame['label'] # Labels (y)
#print(dataframe.head())

# Iterating over K-folds
StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
for train_index, test_index in skf.split(X, y):
    print(train_index, test_index)
    class_labels_count = np.array([len(np.where(y==t)[0]) for t in np.unique(y)])
    print(class_labels_count)
    weights = [0]*(len(np.unique(y))+1)
    for i in range(len(class_labels_count)):   
        weights[i+1] = 1. / class_labels_count[i]
    #print(weights)
    sample_weights = np.array([weights[t] for t in y])

    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    #train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    train_sampler = WeightedRandomSampler(sample_weights, replacement=False, num_samples=len(sample_weights))
    #valid_sampler = WeightedRandomSampler(sample_weights, replacement=True, num_samples=len(sample_weights))

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                               sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    sampler=valid_sampler)

                                              path  label
0              split_Ori_Face/train/LiuWen/534.jpg      1
1              split_Ori_Face/train/LiuWen/253.jpg      1
2              split_Ori_Face/train/LiuWen/432.jpg      1
3              split_Ori_Face/train/LiuWen/334.jpg      1
4              split_Ori_Face/train/LiuWen/412.jpg      1
...                                            ...    ...
30025    split_Ori_Face/train/SavanahGuthrie/3.jpg    200
30026  split_Ori_Face/train/SavanahGuthrie/352.jpg    200
30027  split_Ori_Face/train/SavanahGuthrie/259.jpg    200
30028  split_Ori_Face/train/SavanahGuthrie/232.jpg    200
30029  split_Ori_Face/train/SavanahGuthrie/457.jpg    200

[30030 rows x 2 columns]
                                  path  label
0  split_Ori_Face/train/LiuWen/534.jpg      1
1  split_Ori_Face/train/LiuWen/253.jpg      1
2  split_Ori_Face/train/LiuWen/432.jpg      1
3  split_Ori_Face/train/LiuWen/334.jpg      1
4  split_Ori_Face/train/LiuWen/412.jpg      1
[2

In [6]:

# Track the loss across training
total_loss = []
avg_minibatch_loss = []
N = 50

for epoch in range(50):
    N_minibatch_loss = 0.0
    correct = 0
    total = 0

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):
        print("mini_batch", minibatch_count)
        print('labels', len(labels))
        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        # Perform the forward pass through the network and compute the loss
        outputs = net(images)
        
        loss = criterion(outputs, labels)
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()

        # Update the weights
        optimizer.step()    
        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
               
        
        if minibatch_count % N == 49:
            #Print the loss averaged over the last N mini-batches
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' % (epoch + 1, minibatch_count+1, N_minibatch_loss))
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0

    print("Finished", epoch + 1, "epochs of training")
    # TODO: Implement validation #with torch.no_grad():
    
    with torch.no_grad():
        for minibatch_count, (images, labels) in enumerate(validation_loader, 0):
            # Put the minibatch data in CUDA Tensors and run on the GPU if supported
            images, labels = images.to(computing_device), labels.to(computing_device)
            #print(len(labels))
            for index in range(len(labels)):    
                real_y = labels[index]
                #print('real_y', real_y)
                out = net(images[index].view(-1,3,224,224))
                #print('out:', out)
                pred_y = torch.argmax(out)
                #print('pred_y', pred_y)

                if (real_y.item() == pred_y.item()):
                    correct += 1
                total += 1
    print('Accuracy: %.3f' % ((correct/total)*100))

mini_batch 0
labels 64
mini_batch 1
labels 64
mini_batch 2
labels 64
mini_batch 3
labels 64
mini_batch 4
labels 64
mini_batch 5
labels 64
mini_batch 6
labels 64
mini_batch 7
labels 64
mini_batch 8
labels 64
mini_batch 9
labels 64
mini_batch 10
labels 64
mini_batch 11
labels 64
mini_batch 12
labels 64
mini_batch 13
labels 64
mini_batch 14
labels 64
mini_batch 15
labels 64
mini_batch 16
labels 64
mini_batch 17
labels 64
mini_batch 18
labels 64
mini_batch 19
labels 64
mini_batch 20
labels 64
mini_batch 21
labels 64
mini_batch 22
labels 64
mini_batch 23
labels 64
mini_batch 24
labels 64
mini_batch 25
labels 64
mini_batch 26
labels 64
mini_batch 27
labels 64
mini_batch 28
labels 64
mini_batch 29
labels 64
mini_batch 30
labels 64
mini_batch 31
labels 64
mini_batch 32
labels 64
mini_batch 33
labels 64
mini_batch 34
labels 64
mini_batch 35
labels 64
mini_batch 36
labels 64
mini_batch 37
labels 64
mini_batch 38
labels 64
mini_batch 39
labels 64
mini_batch 40
labels 64
mini_batch 41
labels 64
mi

mini_batch 322
labels 64
mini_batch 323
labels 64
mini_batch 324
labels 64
mini_batch 325
labels 64
mini_batch 326
labels 64
mini_batch 327
labels 64
mini_batch 328
labels 64
mini_batch 329
labels 64
mini_batch 330
labels 64
mini_batch 331
labels 64
mini_batch 332
labels 64
mini_batch 333
labels 64
mini_batch 334
labels 64
mini_batch 335
labels 64
mini_batch 336
labels 64
mini_batch 337
labels 64
mini_batch 338
labels 64
mini_batch 339
labels 64
mini_batch 340
labels 64
mini_batch 341
labels 64
mini_batch 342
labels 64
mini_batch 343
labels 64
mini_batch 344
labels 64
mini_batch 345
labels 64
mini_batch 346
labels 64
mini_batch 347
labels 64
mini_batch 348
labels 64
mini_batch 349
labels 64
Epoch 1, average minibatch 350 loss: 4.087
mini_batch 350
labels 64
mini_batch 351
labels 64
mini_batch 352
labels 64
mini_batch 353
labels 64
mini_batch 354
labels 64
mini_batch 355
labels 64
mini_batch 356
labels 64
mini_batch 357
labels 64
mini_batch 358
labels 64
mini_batch 359
labels 64
mini_ba

mini_batch 172
labels 64
mini_batch 173
labels 64
mini_batch 174
labels 64
mini_batch 175
labels 64
mini_batch 176
labels 64
mini_batch 177
labels 64
mini_batch 178
labels 64
mini_batch 179
labels 64
mini_batch 180
labels 64
mini_batch 181
labels 64
mini_batch 182
labels 64
mini_batch 183
labels 64
mini_batch 184
labels 64
mini_batch 185
labels 64
mini_batch 186
labels 64
mini_batch 187
labels 64
mini_batch 188
labels 64
mini_batch 189
labels 64
mini_batch 190
labels 64
mini_batch 191
labels 64
mini_batch 192
labels 64
mini_batch 193
labels 64
mini_batch 194
labels 64
mini_batch 195
labels 64
mini_batch 196
labels 64
mini_batch 197
labels 64
mini_batch 198
labels 64
mini_batch 199
labels 64
Epoch 2, average minibatch 200 loss: 3.378
mini_batch 200
labels 64
mini_batch 201
labels 64
mini_batch 202
labels 64
mini_batch 203
labels 64
mini_batch 204
labels 64
mini_batch 205
labels 64
mini_batch 206
labels 64
mini_batch 207
labels 64
mini_batch 208
labels 64
mini_batch 209
labels 64
mini_ba

mini_batch 19
labels 64
mini_batch 20
labels 64
mini_batch 21
labels 64
mini_batch 22
labels 64
mini_batch 23
labels 64
mini_batch 24
labels 64
mini_batch 25
labels 64
mini_batch 26
labels 64
mini_batch 27
labels 64
mini_batch 28
labels 64
mini_batch 29
labels 64
mini_batch 30
labels 64
mini_batch 31
labels 64
mini_batch 32
labels 64
mini_batch 33
labels 64
mini_batch 34
labels 64
mini_batch 35
labels 64
mini_batch 36
labels 64
mini_batch 37
labels 64
mini_batch 38
labels 64
mini_batch 39
labels 64
mini_batch 40
labels 64
mini_batch 41
labels 64
mini_batch 42
labels 64
mini_batch 43
labels 64
mini_batch 44
labels 64
mini_batch 45
labels 64
mini_batch 46
labels 64
mini_batch 47
labels 64
mini_batch 48
labels 64
mini_batch 49
labels 64
Epoch 3, average minibatch 50 loss: 3.102
mini_batch 50
labels 64
mini_batch 51
labels 64
mini_batch 52
labels 64
mini_batch 53
labels 64
mini_batch 54
labels 64
mini_batch 55
labels 64
mini_batch 56
labels 64
mini_batch 57
labels 64
mini_batch 58
labels 6

mini_batch 340
labels 64
mini_batch 341
labels 64
mini_batch 342
labels 64
mini_batch 343
labels 64
mini_batch 344
labels 64
mini_batch 345
labels 64
mini_batch 346
labels 64
mini_batch 347
labels 64
mini_batch 348
labels 64
mini_batch 349
labels 64
Epoch 3, average minibatch 350 loss: 2.228
mini_batch 350
labels 64
mini_batch 351
labels 64
mini_batch 352
labels 64
mini_batch 353
labels 64
mini_batch 354
labels 64
mini_batch 355
labels 64
mini_batch 356
labels 64
mini_batch 357
labels 64
mini_batch 358
labels 64
mini_batch 359
labels 64
mini_batch 360
labels 64
mini_batch 361
labels 64
mini_batch 362
labels 64
mini_batch 363
labels 64
mini_batch 364
labels 64
mini_batch 365
labels 64
mini_batch 366
labels 64
mini_batch 367
labels 64
mini_batch 368
labels 64
mini_batch 369
labels 64
mini_batch 370
labels 64
mini_batch 371
labels 64
mini_batch 372
labels 64
mini_batch 373
labels 64
mini_batch 374
labels 64
mini_batch 375
labels 64
mini_batch 376
labels 64
mini_batch 377
labels 64
mini_ba

...

mini_batch 404
labels 64
mini_batch 405
labels 64
mini_batch 406
labels 64
mini_batch 407
labels 64
mini_batch 408
labels 64
mini_batch 409
labels 64
mini_batch 410
labels 64
mini_batch 411
labels 64
mini_batch 412
labels 64
mini_batch 413
labels 64
mini_batch 414
labels 64
mini_batch 415
labels 64
mini_batch 416
labels 64
mini_batch 417
labels 64
mini_batch 418
labels 64
mini_batch 419
labels 64
mini_batch 420
labels 64
mini_batch 421
labels 64
mini_batch 422
labels 64
mini_batch 423
labels 64
mini_batch 424
labels 64
mini_batch 425
labels 64
mini_batch 426
labels 64
mini_batch 427
labels 64
mini_batch 428
labels 64
mini_batch 429
labels 64
mini_batch 430
labels 64
mini_batch 431
labels 64
mini_batch 432
labels 64
mini_batch 433
labels 64
mini_batch 434
labels 64
mini_batch 435
labels 64
mini_batch 436
labels 64
mini_batch 437
labels 64
mini_batch 438
labels 64
mini_batch 439
labels 64
mini_batch 440
labels 64
mini_batch 441
labels 64
mini_batch 442
labels 64
mini_batch 443
labels 64


mini_batch 254
labels 64
mini_batch 255
labels 64
mini_batch 256
labels 64
mini_batch 257
labels 64
mini_batch 258
labels 64
mini_batch 259
labels 64
mini_batch 260
labels 64
mini_batch 261
labels 64
mini_batch 262
labels 64
mini_batch 263
labels 64
mini_batch 264
labels 64
mini_batch 265
labels 64
mini_batch 266
labels 64
mini_batch 267
labels 64
mini_batch 268
labels 64
mini_batch 269
labels 64
mini_batch 270
labels 64
mini_batch 271
labels 64
mini_batch 272
labels 64
mini_batch 273
labels 64
mini_batch 274
labels 64
mini_batch 275
labels 64
mini_batch 276
labels 64
mini_batch 277
labels 64
mini_batch 278
labels 64
mini_batch 279
labels 64
mini_batch 280
labels 64
mini_batch 281
labels 64
mini_batch 282
labels 64
mini_batch 283
labels 64
mini_batch 284
labels 64
mini_batch 285
labels 64
mini_batch 286
labels 64
mini_batch 287
labels 64
mini_batch 288
labels 64
mini_batch 289
labels 64
mini_batch 290
labels 64
mini_batch 291
labels 64
mini_batch 292
labels 64
mini_batch 293
labels 64


mini_batch 104
labels 64
mini_batch 105
labels 64
mini_batch 106
labels 64
mini_batch 107
labels 64
mini_batch 108
labels 64
mini_batch 109
labels 64
mini_batch 110
labels 64
mini_batch 111
labels 64
mini_batch 112
labels 64
mini_batch 113
labels 64
mini_batch 114
labels 64
mini_batch 115
labels 64
mini_batch 116
labels 64
mini_batch 117
labels 64
mini_batch 118
labels 64
mini_batch 119
labels 64
mini_batch 120
labels 64
mini_batch 121
labels 64
mini_batch 122
labels 64
mini_batch 123
labels 64
mini_batch 124
labels 64
mini_batch 125
labels 64
mini_batch 126
labels 64
mini_batch 127
labels 64
mini_batch 128
labels 64
mini_batch 129
labels 64
mini_batch 130
labels 64
mini_batch 131
labels 64
mini_batch 132
labels 64
mini_batch 133
labels 64
mini_batch 134
labels 64
mini_batch 135
labels 64
mini_batch 136
labels 64
mini_batch 137
labels 64
mini_batch 138
labels 64
mini_batch 139
labels 64
mini_batch 140
labels 64
mini_batch 141
labels 64
mini_batch 142
labels 64
mini_batch 143
labels 64


mini_batch 422
labels 64
mini_batch 423
labels 64
mini_batch 424
labels 64
mini_batch 425
labels 64
mini_batch 426
labels 64
mini_batch 427
labels 64
mini_batch 428
labels 64
mini_batch 429
labels 64
mini_batch 430
labels 64
mini_batch 431
labels 64
mini_batch 432
labels 64
mini_batch 433
labels 64
mini_batch 434
labels 64
mini_batch 435
labels 64
mini_batch 436
labels 64
mini_batch 437
labels 64
mini_batch 438
labels 64
mini_batch 439
labels 64
mini_batch 440
labels 64
mini_batch 441
labels 64
mini_batch 442
labels 64
mini_batch 443
labels 64
mini_batch 444
labels 64
mini_batch 445
labels 64
mini_batch 446
labels 64
mini_batch 447
labels 64
mini_batch 448
labels 64
mini_batch 449
labels 64
Epoch 49, average minibatch 450 loss: 0.107
mini_batch 450
labels 64
mini_batch 451
labels 64
mini_batch 452
labels 64
mini_batch 453
labels 64
mini_batch 454
labels 64
mini_batch 455
labels 64
mini_batch 456
labels 64
mini_batch 457
labels 64
mini_batch 458
labels 64
mini_batch 459
labels 64
mini_b

mini_batch 272
labels 64
mini_batch 273
labels 64
mini_batch 274
labels 64
mini_batch 275
labels 64
mini_batch 276
labels 64
mini_batch 277
labels 64
mini_batch 278
labels 64
mini_batch 279
labels 64
mini_batch 280
labels 64
mini_batch 281
labels 64
mini_batch 282
labels 64
mini_batch 283
labels 64
mini_batch 284
labels 64
mini_batch 285
labels 64
mini_batch 286
labels 64
mini_batch 287
labels 64
mini_batch 288
labels 64
mini_batch 289
labels 64
mini_batch 290
labels 64
mini_batch 291
labels 64
mini_batch 292
labels 64
mini_batch 293
labels 64
mini_batch 294
labels 64
mini_batch 295
labels 64
mini_batch 296
labels 64
mini_batch 297
labels 64
mini_batch 298
labels 64
mini_batch 299
labels 64
Epoch 50, average minibatch 300 loss: 0.088
mini_batch 300
labels 64
mini_batch 301
labels 64
mini_batch 302
labels 64
mini_batch 303
labels 64
mini_batch 304
labels 64
mini_batch 305
labels 64
mini_batch 306
labels 64
mini_batch 307
labels 64
mini_batch 308
labels 64
mini_batch 309
labels 64
mini_b